★事前準備★

ご自分の「Googleドライブ」の「MyDrive」に 以下のフォルダ構成と

任意でGoogleColab上の「Stable-Diffusion_WEB-UI（A1111）」で使用したいデータを格納してください。

MyDrive

└StableDiffusion

 　└checkpoints　※使用したいモデルデータを格納

 　└controlnet　※ツール用モデルを格納

 　└embeddings　※EasyNegativeなどを格納

 　└lora　※使用したいLoRAデータを格納

 　└VAE　※使用したいVAEデータを格納

 　└outputs　※生成データのコピー用受け皿フォルダ（フォルダのみでOK）

 ※ローカル環境等で使っているStyleを流用したい場合は、

 マイドライブの「StableDiffusion」フォルダに「styles.csv」のファイルを保存しておいてください。

★このノートブックの使い方★

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～

①まずメニュー項目の「ランタイム」から
 → ランタイムの変更（いずれかのGPUタイプを選択）

②【01】の実行ボタン（▶）を押す、

（▶）の左に✅が付いたら次の（▶）を順番に押していく

※または「ランタイム」から「すべてのセルを実行」を実行する

③【03】の処理が進み、 https://************.gradio.live
と表示されたら、そのURLをクリック

④WEB-UIを停止するときは【03】の（▶）を押して処理をクルクルを停止

⑤再度WEB-UIを使用する際は、【03】の（▶）を押し③と同様

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～


以下、「StableDiffusion_WebUI A1111環境」の起動手順です。

👇【01】Googleドライブのデータをこのノートブックへマウント（接続）

※Googleドライブにある、checkpoint、LoRA、VAE、Embedding、ControlNet用のツールモデル等を参照できるようにする

↓の（▶）を押し、出現するポップアップウィンドウから

　 Googleドライブに接続 → 「アカウントの選択」 → 「アクセスを許可」

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


👇【02】WEB-UIと各種_拡張機能のクローン（起動前にあらかじめ導入）※お好みで増減可能

In [ ]:
# Stable Diffusion Web-UIのクローン
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%cd stable-diffusion-webui

# 最新のWebUIを取得（エラー対策として追加）
!git reset --hard
!git checkout master
!git pull

# 依存関係をインストールする前に、requirements.txt を修正
!sed -i '/pydantic/d' requirements.txt  # 既存の pydantic の記述を削除
!echo -e "\npydantic==1.10.2" >> requirements.txt  # pydanticのバージョンを固定

# 依存関係をインストール
!pip install -r requirements.txt

# wandb を 0.15.12 にダウングレード（pydantic 2.x の要求を避ける）
!pip uninstall -y wandb
!pip install wandb==0.15.12

# TensorFlow を削除（参考コードを元に追加）
!rm -r /usr/local/lib/python3.11/dist-packages/tensorflow*

# Transformers のキャッシュを Google Drive に保存（参考コードから追加）
import os
os.environ['TRANSFORMERS_CACHE'] = "/content/drive/MyDrive/sd_cache/"
os.environ['TORCH_HOME'] = "/content/drive/MyDrive/sd_cache/"

# Colab の Python 警告を無効化（参考コードより）
os.environ['PYTHONWARNINGS'] = 'ignore'
!sed -i 's@text = _formatwarnmsg(msg)@text =""@g' /usr/lib/python3.11/warnings.py
!sed -i 's@raise AttributeError(f"module {module!r} has no attribute {name!r}")@@g' /usr/local/lib/python3.11/dist-packages/jax/_src/deprecations.py

# 拡張機能リポジトリのURLリスト
extensions = [
    "https://github.com/chrisgoringe/Styles-Editor.git",
    "https://github.com/Mikubill/sd-webui-controlnet.git",
    "https://github.com/Bing-su/adetailer.git",
    "https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git",
    "https://github.com/Physton/sd-webui-prompt-all-in-one.git",
    "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git",
    "https://github.com/continue-revolution/sd-webui-segment-anything.git",
    "https://github.com/blue-pen5805/sdweb-easy-prompt-selector.git",
    "https://github.com/AUTOMATIC1111/stable-diffusion-webui-wildcards.git",

    "https://github.com/camenduru/webui-animatediff.git",
    "https://github.com/deforum-art/sd-webui-deforum.git"
]

# 拡張機能のクローン処理
%cd /content/stable-diffusion-webui/extensions
for ext in extensions:
    !git clone {ext}
%cd /content/stable-diffusion-webui

#MP4形式の出力用に「ffpeg」をインストール
%cd /content/stable-diffusion-webui/
!venv\Scripts\activate
!pip install imageio[ffmpeg]

#GoogleドライブへHugingFace、CIVITAIから各種必要なモデルデータをDL
%cd /content/stable-diffusion-webui/
#!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd14_v1.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/webui-animatediff/model
#!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v1.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/webui-animatediff/model
#!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v2.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/webui-animatediff/model
!wget https://huggingface.co/conrevo/AnimateDiff-A1111/resolve/main/motion_module/mm_sd15_v3.safetensors --directory-prefix=/content/stable-diffusion-webui/extensions/webui-animatediff/model


👇【03】WEB-UIの起動　（各種大容量データはGoogleドライブを参照する設定）

処理が進み、 https://****.gradio.live と表示されたら、そのURLをクリック

※生成した画像は自動的に/content/drive/MyDrive/StableDiffusion/outputs へ保存されます

※起動オプション追記もお好みで可能です

★ ↓の（▶）で WEB-UIの「起動」「停止」「再起動」を操作します。

編集するにはダブルクリックするか Enter キーを押してください

In [ ]:
# 出力ディレクトリを設定するスクリプトを作成
script_content = """
import os
from modules import shared

# Google Driveの出力ディレクトリを設定
shared.opts.data['outdir_txt2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-images'
shared.opts.data['outdir_txt2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-grids'
shared.opts.data['outdir_img2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-images'
shared.opts.data['outdir_img2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-grids'
shared.opts.data['outdir_extras_images'] = '/content/drive/MyDrive/StableDiffusion/outputs/extras-images'

print("出力ディレクトリの設定が完了しました。")
"""
# スクリプトを保存
script_path = "/content/stable-diffusion-webui/scripts/set_output_dir.py"
with open(script_path, "w") as f:
    f.write(script_content)

#style.csvファイルコピー ※MyDrive/StableDiffusionに保存したファイルをコピーして、登録済みのStyleをForgeで使用可能にします
!cp "/content/drive/MyDrive/StableDiffusion/styles.csv" "/content/stable-diffusion-webui"

# WEB-UIの起動
!python launch.py --share --no-half-vae --disable-nan-check \
  --ckpt-dir "/content/drive/MyDrive/StableDiffusion/checkpoints" \
  --embeddings-dir "/content/drive/MyDrive/StableDiffusion/embeddings" \
  --lora-dir "/content/drive/MyDrive/StableDiffusion/lora" \
  --vae-dir "/content/drive/MyDrive/StableDiffusion/vae" \
  --controlnet-dir "/content/drive/MyDrive/StableDiffusion/controlnet"
